#### Install required dependencies for LlamaIndex and Elasticsearch

In [ ]:
!pip install llama-index
!pip install llama-index-cli
!pip install llama-index-core
!pip install llama-index-embeddings-elasticsearch
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-ollama
!pip install llama-index-indices-managed-llama-cloud
!pip install llama-index-legacy
!pip install llama-index-llms-ollama
!pip install llama-index-readers-elasticsearch
!pip install llama-index-readers-file
!pip install llama-index-readers-llama-parse
!pip install llama-index-vector-stores-elasticsearch
!pip install llama-parse
!pip install llamaindex-py-client


#### Import packages

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.vector_stores.elasticsearch import ElasticsearchStore
from llama_index.core import VectorStoreIndex, QueryBundle
from llama_index.llms.ollama import Ollama
from llama_index.core import Document, Settings
from getpass import getpass
from urllib.request import urlopen
import json

#### Prompt user to provide Cloud ID and API Key

In [ ]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

#### Prepare documents for chunking and ingestion

In [ ]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/datasets/workplace-documents.json"

response = urlopen(url)
workplace_docs = json.loads(response.read())


# Building Document required by LlamaIndex.
documents = [Document(text=doc['content'],
                          metadata={"name": doc['name'],"summary": doc['summary'],"rolePermissions": doc['rolePermissions']})
                 for doc in workplace_docs]

#### Define Elasticsearch and ingest pipeline in LlamaIndex for document processing. Use Llama3 for generating embeddings.

In [ ]:
es_vector_store = ElasticsearchStore(index_name="workplace_index",
                                     vector_field='content_vector',
                                     text_field='content',
                                     es_cloud_id=ELASTIC_CLOUD_ID,
                                     es_api_key=ELASTIC_API_KEY)
# Embedding Model to do local embedding using Ollama.
ollama_embedding = OllamaEmbedding("llama3")
# LlamaIndex Pipeline configured to take care of chunking, embedding
# and storing the embeddings in the vector store.
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=512, chunk_overlap=100),
        ollama_embedding
    ], vector_store=es_vector_store
)

#### Execute pipeline, which will chunk the data, generate embeddings using Llama3 and ingest into Elasticsearch index, with embeddings in a dense vector field.

In [ ]:
pipeline.run(show_progress=True,documents=documents)

#### Define LLM settings. This connects to your local LLM. Please refer to https://ollama.com/library/llama3 for details on steps to run Llama3 locally. 
#### If you have sufficient resources (atleast >64 GB Ram and GPU available) then you could try the 70B parameter version of Llama3. 

In [ ]:
Settings.embed_model = ollama_embedding
local_llm = Ollama(model="llama3")

### Setup Semantic search and integrate with Llama3. 

In [ ]:
index = VectorStoreIndex.from_vector_store(es_vector_store)
query_engine = index.as_query_engine(local_llm, similarity_top_k=10)

# Customer Query
query = "What are the organizations sales goals?"
bundle = QueryBundle(query_str=query,
embedding=Settings.embed_model.get_query_embedding(query=query))

response = query_engine.query(bundle)

print(response.response)